In [1]:
# import all the required packages and modules
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 
from matplotlib import pyplot as plt
import requests, json
import numpy as np
import sys
import os

# path to where you put the metatlas.py file:
sys.path.append( '/Users/bpb/Data/programming/MetaboliteAtlas/github/metatlas/' )
import metatlas
import re
import csv

In [2]:
client = requests.Session()
client = metatlas.authenticateUser(client,'bpb')

#### edit the myExperimentID string ####
myExperimentID = '5524364cf207834b9656cd97'
# Get the entries for a specific dictionary
dictId = '55256a417626400162911485'

#Get the Atlas to make sure you properly authenticated and are authorized
url = 'https://metatlas.nersc.gov/api/dict/%s/' % dictId
r = client.get(url)
dictData = json.loads(r.text)
print dictData.keys()

········
[u'name', u'compounds', u'creator', u'creation_date', u'sample', u'_id', u'method']


In [5]:
# # Share the experiment and dictionary with another user.
# payload = {"user":'username',"perms":["read","write"]}
# sendData=json.dumps(payload)
# url = 'https://metatlas.nersc.gov/api/experiment/%s/share/' % myExperimentID
# r = client.post(url, data=sendData)
# print r.content
# url = 'https://metatlas.nersc.gov/api/dict/%s/share/' % dictId
# r = client.post(url, data=sendData)
# print r.content

In [6]:
# url = 'https://metatlas.nersc.gov/api/experiment/%s' % myExperimentID
# r = client.get(url)
# files = json.loads(r.content)
# fileInfo = {'fid':[],'name':[],'status':[]};
# fid = open('fileInfo_herlperFile.tab','wb')
# fid.write('index\tstatus\tname\tfid\tpolarity\tgroup\tinclusion_order\tnormalization_factor\tretention_correction\n')
# for i,myRun in enumerate(files[u'runs']):
#     splitPathToFile = os.path.split(myRun[u'in_file'])
#     fid.write('%d\t%d\t%s\t%d\tpos\tgroup1\n' % (i,myRun[u'pending'],splitPathToFile[1],myRun[u'_id'][u'file_id']))
#     if myRun[u'pending'] == 0:
#         fileInfo['fid'].append(myRun[u'_id'][u'file_id'])
#         fileInfo['name'].append(splitPathToFile[1])
#         fileInfo['status'].append(myRun[u'pending']) #only keep if status is 0
# pathYouWant = splitPathToFile[0] # TODO: we will have to see what this will do on a window's computer.  taking a linux path and using os.
# fid.close()

In [6]:
# import the fileInfo sheet with annotated group information, polarity, and plot order
url = 'https://metatlas.nersc.gov/api/experiment/%s' % myExperimentID
r = client.get(url)
files = json.loads(r.content)
myArray = files[u'runs'][0][u'_id'][u'array_name']

filename = 'caj_fileInfo_practice.txt'
with open(filename,'rU') as file_object:
    newfileInfo = list(csv.DictReader(file_object, dialect='excel-tab'))
keys = newfileInfo[0].iterkeys()
fileInfo = {key: [d[key] for d in newfileInfo] for key in keys}
fileInfo['fid'] = map(int, fileInfo['fid'])
fileInfo['index'] = map(int, fileInfo['index'])
fileInfo['inclusion_order'] = map(int, fileInfo['inclusion_order'])
fileInfo['status'] = map(int, fileInfo['status'])
fileInfo['normalization_factor'] = map(float, fileInfo['normalization_factor'])
# fileInfo['retention_correction'] = map(float, fileInfo['retention_correction'])

idx = np.argsort(fileInfo['inclusion_order'])
export_fileIds = np.asarray(fileInfo['fid'])[idx]

In [9]:
# #add compounds to an atlas from a well formated spreadsheet that are new and
# #update any that were already there, but have changed
# # filename = 'example_metatlas_spreadsheet.txt'
# filename = 'metatlas_spreadsheet_CN20150406.txt'
# with open(filename,'rU') as file_object:
#     sheetData = list(csv.DictReader(file_object, dialect='excel-tab'))

# url = 'https://metatlas.nersc.gov/api/dict/%s/' % dictId
# r = client.get(url)
# dictData = json.loads(r.text)

# for compound in sheetData:
# #     print compound
#     cID = filter( lambda x: x[u'name']==compound['name'], dictData[u'compounds'])
# #     print cID
#     if not cID:
#         # a new entry is created if that compound name doesn't exist
#         url = 'https://metatlas.nersc.gov/api/dict/%s/' % dictId
#         print compound
#         r = client.post(url, data=json.dumps([compound]))
#         print(r.text)
#     else:
#         # edit the entry if it already exists
#         url = 'https://metatlas.nersc.gov/api/compound/%s/' % cID[0][u'_id']
#         r = client.patch(url, data=json.dumps(compound))
#         print(r.text)
# url = 'https://metatlas.nersc.gov/api/dict/%s/' % dictId
# r = client.get(url)
# dictData = json.loads(r.text)  

In [7]:
import time
polarity = 1
extraTime = 0.3
data = [[] for i in range(len(dictData[u'compounds']))]
myList = ','.join(map(str, export_fileIds))
print myList
for i,compound in enumerate(dictData[u'compounds']):
    for j in range(7):
        temp = compound.copy()
        temp['mz'] = float(temp['mz']) + float(j)*1.003355 #this isn't really right because we have deuterium not carbon
        data[i].append(metatlas.getEICForCompounds(temp,myArray,export_fileIds,extraTime,client,polarity))
        time.sleep(1)
    print i


28499,28488
0
1
2
3
4


In [8]:
# pickle the data incase you need it offline
metatlas_data = {'dictId':dictId,'myExperimentID':myExperimentID,'dictData':dictData, 'data':data,'fileInfo':fileInfo,'myList':myList,'export_fileIds':export_fileIds}
import pickle
pickle.dump( metatlas_data, open( "50ppm_caj_data_practice_metatas_isotopes.pkl", "wb" ) )